<a href="https://colab.research.google.com/github/sam-msds/Final_Project/blob/main/PreProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data PreProcessing

In [ ]:
!pip install diffusers==0.20.2

In [ ]:
# Upgrade numpy to fix binary incompatibility
!pip install --upgrade --force-reinstall numpy

# Restart the runtime to apply changes (you'll need to rerun everything after this)
import os
os.kill(os.getpid(), 9)



  Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.5
    Uninstalling numpy-2.2.5:
      Successfully uninstalled numpy-2.2.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 1.5.3 which is incompatible.
plotnine 0.14.5 requires pandas>=2.2.0, but you have pandas 1.5.3 which is incompatible.
xarray 2025.3.1 requires pandas>=2.1, but you have pandas 1.5.3 which is incompatible.
dask-expr 1.1.21 requires pandas>=2, but you have pandas 1.5.3 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.5 which is incompatible.
mizani 0.13.3 requires pandas

Connect To Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install clean_text
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.0 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171031 sha256=7212f3351668ca5bec8a7159c9b1372bc040d838e4115241015a94ca242fabef
  Stored in directory: /root/.cache/pip/wheels/bd/22/e5/b69726d5e1a19795ecd3b3e7464b16c0f1d019aa94ff1c8578
Successfully built emoji


***Create a submit of mimic-iii data. Due to large data volume it's not feasable to work with whole dataset.***

In [ ]:

# Because of the large size of the data, the following code was run locally to
# generate a small subsample of the data that was uploaded to Google Drive
# for demonstration purposes.
import pandas as pd

base_path = '/content/drive/MyDrive/CS598DLH_KeyClass_Reproduce/'

#diagnoses_icd = pd.read_csv(base_path + 'DIAGNOSES_ICD.csv', usecols=['hadm_id', 'icd9_code']).dropna()
#note_events = pd.read_csv(base_path + 'NOTEEVENTS.csv', usecols=['subject_id', 'hadm_id', 'text','category'])
note_events = pd.read_csv(base_path + 'NOTEEVENTS.csv')
note_events = note_events[note_events['CATEGORY'] == 'Discharge summary']

#output = pd.merge(diagnoses_icd, note_events, on='hadm_id', how='inner').drop_duplicates()
output = note_events.sample(50000).to_csv(base_path + 'mimic_subset.csv', index=False)



<ipython-input-3-c9f2b86bfe22>:10: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  note_events = pd.read_csv(base_path + 'NOTEEVENTS.csv')


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import random
from datetime import datetime
from google.colab import drive

# Mount Google Drive to access files
drive.mount('/content/drive')

# Define file paths on Google Drive (Update these paths with your Google Drive file locations)
notes_file_path = '/content/drive/MyDrive/CS598DLH_KeyClass_Reproduce/mimic_subset.csv'
diagnoses_file_path = '/content/drive/MyDrive/CS598DLH_KeyClass_Reproduce/DIAGNOSES_ICD.csv'
output_file_path = '/content/drive/MyDrive/CS598DLH_KeyClass_Reproduce/icd9NotesDataTable.csv'
train_file_path = '/content/drive/MyDrive/CS598DLH_KeyClass_Reproduce/icd9NotesDataTable_train.csv'
valid_file_path = '/content/drive/MyDrive/CS598DLH_KeyClass_Reproduce/icd9NotesDataTable_valid.csv'

# Import necessary libraries
import pandas as pd
import numpy as np
import random
from datetime import datetime
from google.colab import drive

# ... (rest of your code) ...

# Define function for extracting Level 2 and Top Level ICD codes
def get_level2_and_toplevel(icd9_codes):
    level2_list = []
    toplevel_list = []

    for icd9 in icd9_codes:
        if icd9 == '' or pd.isna(icd9) or icd9[:3] == 'nan': # Check for empty, NaN, and 'nan' strings
            continue
        if icd9.startswith('E'):
            level2 = icd9[:4]
            toplevel = "cat:19"
        elif icd9.startswith('V'):
            level2 = icd9[:3]
            toplevel = "cat:19"
        else:
            try:  # Added a try-except block to handle potential errors
                icd9_num = int(icd9[:3])
            except ValueError:
                print(f"Warning: Could not convert '{icd9[:3]}' to integer. Skipping this code.")
                continue  # Skip to the next code if conversion fails

            if 1 <= icd9_num <= 139:
                toplevel = "cat:1"
            elif 140 <= icd9_num <= 239:
                toplevel = "cat:2"
            elif 240 <= icd9_num <= 279:
                toplevel = "cat:3"
            elif 280 <= icd9_num <= 289:
                toplevel = "cat:4"
            elif 290 <= icd9_num <= 319:
                toplevel = "cat:5"
            elif 320 <= icd9_num <= 359:
                toplevel = "cat:6"
            elif 360 <= icd9_num <= 389:
                toplevel = "cat:7"
            elif 390 <= icd9_num <= 459:
                toplevel = "cat:8"
            elif 460 <= icd9_num <= 519:
                toplevel = "cat:9"
            elif 520 <= icd9_num <= 579:
                toplevel = "cat:10"
            elif 580 <= icd9_num <= 629:
                toplevel = "cat:11"
            elif 630 <= icd9_num <= 679:
                toplevel = "cat:12"
            elif 680 <= icd9_num <= 709:
                toplevel = "cat:13"
            elif 710 <= icd9_num <= 739:
                toplevel = "cat:14"
            elif 740 <= icd9_num <= 759:
                toplevel = "cat:15"
            elif 760 <= icd9_num <= 779:
                toplevel = "cat:16"
            elif 780 <= icd9_num <= 799:
                toplevel = "cat:17"
            elif 800 <= icd9_num <= 999:
                toplevel = "cat:18"
            level2 = icd9[:3]

        level2_list.append(level2)
        toplevel_list.append(toplevel)

    return '-'.join(sorted(set(level2_list))), '-'.join(sorted(set(toplevel_list)))


# Load in Data
notes = pd.read_csv(notes_file_path)
diagnoses = pd.read_csv(diagnoses_file_path)

# Collapse all ICD9 codes for each admission
diagnoses = diagnoses[['HADM_ID', 'SUBJECT_ID', 'ICD9_CODE']]
# Convert 'ICD9_CODE' to string before aggregation
diagnoses['ICD9_CODE'] = diagnoses['ICD9_CODE'].astype(str)
diagnoses = diagnoses.groupby(['HADM_ID', 'SUBJECT_ID']).agg({'ICD9_CODE': lambda x: '-'.join(x)}).reset_index()


# Filter Notes for "Discharge summary"
notes = notes[notes['CATEGORY'] == "Discharge summary"]

# Remove duplicates by keeping the first entry for each HADM_ID
notes['CHARTDATE'] = pd.to_datetime(notes['CHARTDATE'])
notes['HADM_ID'] = notes['HADM_ID'].astype(int)
notes = notes.groupby('HADM_ID').first().reset_index()

# Merge diagnoses and notes data
icd9_notes_data = pd.merge(diagnoses, notes, on='HADM_ID', how='right')

# Remove empty ICD9 codes
icd9_notes_data = icd9_notes_data[icd9_notes_data['ICD9_CODE'] != '']

# Apply Level2ICD and TopLevelICD extraction
icd9_notes_data[['Level2ICD', 'TopLevelICD']] = icd9_notes_data['ICD9_CODE'].apply(
    lambda x: pd.Series(get_level2_and_toplevel(x.split('-')))
)

# Split into Training and Validation sets
training_frac = 0.75
train_idx = random.sample(range(len(icd9_notes_data)), int(len(icd9_notes_data) * training_frac))
icd9_notes_data_train = icd9_notes_data.iloc[train_idx]
icd9_notes_data_valid = icd9_notes_data.drop(train_idx)

# Write to file on Google Drive
icd9_notes_data.to_csv(output_file_path, index=False)
icd9_notes_data_train.to_csv(train_file_path, index=False)
icd9_notes_data_valid.to_csv(valid_file_path, index=False)

# Print the first few rows for inspection
print(icd9_notes_data.head())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-4-8ee357143ad5>:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diagnoses['ICD9_CODE'] = diagnoses['ICD9_CODE'].astype(str)


   HADM_ID  SUBJECT_ID_x                                          ICD9_CODE  \
0   100001         58526  25013-3371-5849-5780-V5867-25063-5363-4580-250...   
1   100003         54610  53100-2851-07054-5715-45621-53789-4019-53550-7823   
2   100006          9895   49320-51881-486-20300-2761-7850-3090-V1251-V1582   
3   100007         23018                           56081-5570-9973-486-4019   
4   100010         55853                          1890-1961-1987-1976-27652   

   ROW_ID  SUBJECT_ID_y  CHARTDATE  CHARTTIME  STORETIME           CATEGORY  \
0   42102         58526 2117-09-17        NaN        NaN  Discharge summary   
1   19215         54610 2150-04-21        NaN        NaN  Discharge summary   
2   55972          9895 2108-04-18        NaN        NaN  Discharge summary   
3   50238         23018 2145-04-07        NaN        NaN  Discharge summary   
4   40054         55853 2109-12-14        NaN        NaN  Discharge summary   

  DESCRIPTION  CGID  ISERROR  \
0      Report   Na

In [ ]:
import pandas as pd

# Importing the data as dataframes
train_df = pd.read_csv('/content/drive/MyDrive/CS598DLH_KeyClass_Reproduce/icd9NotesDataTable_train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/CS598DLH_KeyClass_Reproduce/icd9NotesDataTable_valid.csv')

In [ ]:
#Printing the Dataset Statistics
print("size(Train) = ",len(train_df))
print("size(Test) = ",len(test_df))

size(Train) =  33741
size(Test) =  11248


In [ ]:
# Converting the ICD-9 Codes into a multi-hot vector and adding it to the dataframe
# Also, finding the number of data points for each category

labels = []
train_map = {}
for code in train_df["TopLevelICD"]:
  # Check if 'code' is a string before applying split
  if isinstance(code, str):
    row = [0]*19
    splitstring = code.split('-')
    for cat in splitstring:
      val = int(cat.split(':')[1])-1
      row[val] = 1
      if val not in train_map:
        train_map[val] = 0
      train_map[val]+=1
    labels.append(row)
  else: # If 'code' is not a string (e.g., NaN), append a default value
    labels.append([0]*19)  # Append a list of 19 zeros as a default value

print(labels)

train_df["Code"] = labels
labels = []
test_map = {}
for code in test_df["TopLevelICD"]:
  # Check if 'code' is a string before applying split
  if isinstance(code, str):
    row = [0]*19
    splitstring = code.split('-')
    for cat in splitstring:
      val = int(cat.split(':')[1])-1
      row[val] = 1
      if val not in test_map:
        test_map[val] = 0
      test_map[val]+=1
    labels.append(row)
  else: # If 'code' is not a string (e.g., NaN), append a default value
    labels.append([0]*19) # Append a list of 19 zeros as a default value

test_df["Code"] = labels
print(labels)


[[1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1], [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1], [0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1], [0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1], [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1], [0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
train_df.head(5)
train_df["TEXT"]

,TEXT
0,Admission Date: [**2166-10-23**] ...
1,Admission Date: [**2175-10-12**] ...
2,Admission Date: [**2142-6-20**] ...
3,Unit No: [**Numeric Identifier 72923**]\nAdmi...
4,Admission Date: [**2183-6-11**] ...
...,...
33736,Admission Date: [**2156-12-19**] Discharg...
33737,Admission Date: [**2159-9-3**] D...
33738,Admission Date: [**2195-1-30**] Dischar...
33739,Admission Date: [**2165-4-23**] ...


In [ ]:
test_df.head()

,HADM_ID,SUBJECT_ID_x,ICD9_CODE,ROW_ID,SUBJECT_ID_y,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT,Level2ICD,TopLevelICD,Code
0,100001,58526,25013-3371-5849-5780-V5867-25063-5363-4580-250...,42102,58526,2117-09-17,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2117-9-11**] ...,250-337-362-403-458-536-578-584-585-707-V13-V58,cat:10-cat:11-cat:13-cat:19-cat:3-cat:6-cat:7-...,"[0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, ..."
1,100021,29971,E8788-E8497-4019-04104-0413-5728-V1581-2819-78...,31610,29971,2109-10-16,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2109-8-17**] ...,041-281-287-291-303-304-401-427-507-518-571-57...,cat:1-cat:10-cat:11-cat:17-cat:18-cat:19-cat:4...,"[1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, ..."
2,100025,4528,V3000-7756-99811-7708-7661-V502-E8788-V053,14579,4528,2191-07-15,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2191-7-13**] Dischar...,766-770-775-998-E878-V05-V30-V50,cat:16-cat:18-cat:19,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,100028,53456,5761-0389-5184-57481-99591-42731-2875-8738-E9288,41033,53456,2142-12-30,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2142-12-23**] ...,038-287-427-518-574-576-873-995-E928,cat:1-cat:10-cat:18-cat:19-cat:4-cat:8-cat:9,"[1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, ..."
4,100030,12803,41519-486-4510-2761-56881-2859,9785,12803,2199-12-19,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2199-12-3**] Dischar...,276-285-415-451-486-568,cat:10-cat:3-cat:4-cat:8-cat:9,"[0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, ..."


In [ ]:
import pprint

print("Training Data Map (category ID to number of data points : ")
pprint.pprint(train_map) # Note - the IDs are 0 indexed
print("Testing Data Map (category ID to number of data points : ")
pprint.pprint(test_map) # Note - the IDs are 0 indexed
print("Total number of categories = ",len(train_map.keys()))

Training Data Map (category ID to number of data points : 
{0: 9077,
 1: 5409,
 2: 22121,
 3: 12176,
 4: 10101,
 5: 8557,
 6: 2378,
 7: 26484,
 8: 15722,
 9: 13145,
 10: 13574,
 11: 92,
 12: 3825,
 13: 6289,
 14: 1775,
 15: 2373,
 16: 12498,
 17: 13949,
 18: 23236}
Testing Data Map (category ID to number of data points : 
{0: 3030,
 1: 1837,
 2: 7404,
 3: 3972,
 4: 3369,
 5: 2811,
 6: 803,
 7: 8847,
 8: 5192,
 9: 4284,
 10: 4583,
 11: 30,
 12: 1273,
 13: 2050,
 14: 574,
 15: 779,
 16: 4139,
 17: 4679,
 18: 7742}
Total number of categories =  19


In [ ]:
from google.colab import drive
import pandas as pd
import os
import re

# Mount Google Drive
drive.mount('/content/drive')

# Set your working directory in Drive
drive_path = "/content/drive/MyDrive/CS598DLH_KeyClass_Reproduce/"
#os.makedirs(drive_path, exist_ok=True)

# Example: Assume you already have train_df loaded or read it from a CSV
# train_df = pd.read_csv(drive_path + "train_data.csv")


# Write TEXT to train.txt
with open(os.path.join(drive_path, "train.txt"), "w", encoding="utf-8") as f:
    for line in train_df["TEXT"]:
        clean_line = re.sub(r"\s+", " ", str(line)).strip()  # Replace newlines/tabs/multiple spaces with a single space
        f.write(clean_line + "\n")

# Write CODE to train_labels.txt without brackets, one line per entry
with open(os.path.join(drive_path, "train_labels.txt"), "w", encoding="utf-8") as f:
    for label in train_df["Code"]:
        if isinstance(label, list):
            line = ",".join(str(x) for x in label)
        else:
            line = str(label)
        f.write(line.strip() + "\n")




# Write TEXT to train.txt
with open(os.path.join(drive_path, "test.txt"), "w", encoding="utf-8") as f:
    for line in test_df["TEXT"]:
        clean_line = re.sub(r"\s+", " ", str(line)).strip()  # Replace newlines/tabs/multiple spaces with a single space
        f.write(clean_line + "\n")

# Write CODE to train_labels.txt without brackets, one line per entry
with open(os.path.join(drive_path, "test_labels.txt"), "w", encoding="utf-8") as f:
    for label in train_df["Code"]:
        if isinstance(label, list):
            line = ",".join(str(x) for x in label)
        else:
            line = str(label)
        f.write(line.strip() + "\n")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

# Assume the label column is called 'label'
# Split each label string by '-', flatten into a single list
all_labels = train_df['TopLevelICD'].dropna().apply(lambda x: x.split('-'))
flattened_labels = [label for sublist in all_labels for label in sublist]

# Count each unique label
from collections import Counter
label_counts = Counter(flattened_labels)

# Convert to a DataFrame for better display
label_dist_df = pd.DataFrame.from_dict(label_counts, orient='index', columns=['count']).sort_values('count', ascending=False)

print("Class-specific data distribution:")
print(label_dist_df)

Class-specific data distribution:
        count
cat:8   26484
cat:19  23236
cat:3   22121
cat:9   15722
cat:18  13949
cat:11  13574
cat:10  13145
cat:17  12498
cat:4   12176
cat:5   10101
cat:1    9077
cat:6    8557
cat:14   6289
cat:2    5409
cat:13   3825
cat:7    2378
cat:16   2373
cat:15   1775
cat:12     92
